In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import warnings
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [2]:
start=dt.datetime(2025,1,1)
end=dt.datetime(2025,7,15)
stock_data=yf.download('WIPRO.NS',start=start,end=end)

C:\Users\DELL\AppData\Local\Temp\ipykernel_12368\2514622363.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data=yf.download('WIPRO.NS',start=start,end=end)
[*********************100%***********************]  1 of 1 completed


In [3]:
stock_data

Price,Close,High,Low,Open,Volume
Ticker,WIPRO.NS,WIPRO.NS,WIPRO.NS,WIPRO.NS,WIPRO.NS
Date,,,,,
2025-01-01,288.769043,291.701946,287.470904,288.480574,7878201
2025-01-02,292.134674,292.759739,285.740027,286.557394,8800435
2025-01-03,283.143707,292.134662,282.710975,291.653861,12535720
2025-01-06,282.999451,286.365058,279.393443,283.672584,9173687
2025-01-07,282.759064,286.076602,280.114658,282.999464,7381943
...,...,...,...,...,...
2025-07-08,264.448395,265.085850,262.192774,262.634076,5519016
2025-07-09,262.634094,264.203236,259.741018,262.830248,11564433


In [4]:
stock_data.columns=stock_data.columns.droplevel(1)
stock_data

Price,Close,High,Low,Open,Volume
Date,,,,,
2025-01-01,288.769043,291.701946,287.470904,288.480574,7878201
2025-01-02,292.134674,292.759739,285.740027,286.557394,8800435
2025-01-03,283.143707,292.134662,282.710975,291.653861,12535720
2025-01-06,282.999451,286.365058,279.393443,283.672584,9173687
2025-01-07,282.759064,286.076602,280.114658,282.999464,7381943
...,...,...,...,...,...
2025-07-08,264.448395,265.085850,262.192774,262.634076,5519016
2025-07-09,262.634094,264.203236,259.741018,262.830248,11564433
2025-07-10,259.937134,262.388908,258.073791,262.388908,6911031


In [5]:
stock_data.drop('Volume',axis=1,inplace=True)
stock_data

Price,Close,High,Low,Open
Date,,,,
2025-01-01,288.769043,291.701946,287.470904,288.480574
2025-01-02,292.134674,292.759739,285.740027,286.557394
2025-01-03,283.143707,292.134662,282.710975,291.653861
2025-01-06,282.999451,286.365058,279.393443,283.672584
2025-01-07,282.759064,286.076602,280.114658,282.999464
...,...,...,...,...
2025-07-08,264.448395,265.085850,262.192774,262.634076
2025-07-09,262.634094,264.203236,259.741018,262.830248
2025-07-10,259.937134,262.388908,258.073791,262.388908


In [6]:
start_date=dt.datetime(2025,1,1)
end_date=dt.datetime(2025,5,13)
all_date=pd.date_range(start_date,end_date,freq='d')
stock_data.index=all_date
stock_data

Price,Close,High,Low,Open
2025-01-01,288.769043,291.701946,287.470904,288.480574
2025-01-02,292.134674,292.759739,285.740027,286.557394
2025-01-03,283.143707,292.134662,282.710975,291.653861
2025-01-04,282.999451,286.365058,279.393443,283.672584
2025-01-05,282.759064,286.076602,280.114658,282.999464
...,...,...,...,...
2025-05-09,264.448395,265.085850,262.192774,262.634076
2025-05-10,262.634094,264.203236,259.741018,262.830248
2025-05-11,259.937134,262.388908,258.073791,262.388908
2025-05-12,253.317337,256.994999,252.777959,255.965266


In [7]:
dftest=adfuller(stock_data['Close'],autolag='AIC')
dfoutput=pd.Series(dftest[0:4],index=('Test Statistic','p-value','#lags used','#observations used'))
for key,value in dftest[4].items():
    dfoutput['Critical value (%s)' %key]=value

dfoutput

Test Statistic           -1.287073
p-value                   0.635066
#lags used                0.000000
#observations used      132.000000
Critical value (1%)      -3.480888
Critical value (5%)      -2.883697
Critical value (10%)     -2.578586
dtype: float64

In [8]:
print('Condition for stationarity using ADF:')
print('If Test Statistic < Critical value and p-value < 0.05 --> Reject Null Hypothesis --> Stationary data')
print('Else --> Accept Null Hypothesis --> Non-Stationary data')

if (
    (dfoutput['Test Statistic'] < dfoutput ['Critical value (1%)'] or 
     dfoutput['Test Statistic'] < dfoutput ['Critical value (5%)'] or 
     dfoutput['Test Statistic'] < dfoutput ['Critical value (10%)']) and 
     dfoutput['p-value'] < 0.05):
    print('\nCondition: Test Statistic < Critical value and p-value < 0.05 is true. Stationary as per ADF')
else:
    print('\nCondition: Test Statistic < Critical value and p-value < 0.05 is not true. Non- Stationary as per ADF')

Condition for stationarity using ADF:
If Test Statistic < Critical value and p-value < 0.05 --> Reject Null Hypothesis --> Stationary data
Else --> Accept Null Hypothesis --> Non-Stationary data

Condition: Test Statistic < Critical value and p-value < 0.05 is not true. Non- Stationary as per ADF


In [9]:
ms=MinMaxScaler()
prestock_data=ms.fit_transform(stock_data)

In [10]:
prestock_data

array([[0.71023568, 0.69510017, 0.7413241 , 0.70944197],
       [0.75065773, 0.70767125, 0.72123392, 0.68739578],
       [0.64267404, 0.70024268, 0.68607591, 0.74581863],
       [0.64094148, 0.6316752 , 0.64756953, 0.65432604],
       [0.63805438, 0.62824712, 0.65594063, 0.6466098 ],
       [0.67847605, 0.63910372, 0.63361804, 0.63227966],
       [0.61784317, 0.63396076, 0.65649858, 0.67637234],
       [0.71312315, 0.69395751, 0.6894242 , 0.64660962],
       [0.6126466 , 0.64824597, 0.64422142, 0.69786807],
       [0.62419576, 0.59453465, 0.60292472, 0.61905178],
       [0.62188556, 0.59624888, 0.63808271, 0.63227956],
       [0.56875981, 0.61681907, 0.60459901, 0.66424691],
       [0.49831106, 0.51739627, 0.5353994 , 0.57495899],
       [0.70965877, 0.71852769, 0.71900198, 0.73148881],
       [0.68771538, 0.72367017, 0.72848856, 0.73589782],
       [0.81186825, 0.77109634, 0.74578903, 0.70227742],
       [0.91119026, 0.93794406, 0.87191097, 0.83180029],
       [0.93890786, 0.91394525,

In [11]:
prestock_data.shape

(133, 4)

In [12]:
stock_table=pd.DataFrame(prestock_data,columns=['Close','High','Low','Open'])
stock_table

,Close,High,Low,Open
0,0.710236,0.695100,0.741324,0.709442
1,0.750658,0.707671,0.721234,0.687396
2,0.642674,0.700243,0.686076,0.745819
3,0.640941,0.631675,0.647570,0.654326
4,0.638054,0.628247,0.655941,0.646610
...,...,...,...,...
128,0.418139,0.378788,0.447922,0.413153
129,0.396349,0.368298,0.419465,0.415402
130,0.363957,0.346736,0.400114,0.410343
131,0.284452,0.282634,0.338645,0.336706


In [13]:
warnings.filterwarnings('ignore')

In [14]:
performance={'Model':[],'RMSE':[],'MaPe':[],'Lag':[],'Test':[]}
list=['Close','High','Low','Open']

In [15]:
dataset=stock_table[list]
dataset

,Close,High,Low,Open
0,0.710236,0.695100,0.741324,0.709442
1,0.750658,0.707671,0.721234,0.687396
2,0.642674,0.700243,0.686076,0.745819
3,0.640941,0.631675,0.647570,0.654326
4,0.638054,0.628247,0.655941,0.646610
...,...,...,...,...
128,0.418139,0.378788,0.447922,0.413153
129,0.396349,0.368298,0.419465,0.415402
130,0.363957,0.346736,0.400114,0.410343
131,0.284452,0.282634,0.338645,0.336706


In [16]:
test_obs=30
train_data=dataset[:-test_obs]
test_data=dataset[-test_obs:]

In [17]:
train_data

,Close,High,Low,Open
0,0.710236,0.695100,0.741324,0.709442
1,0.750658,0.707671,0.721234,0.687396
2,0.642674,0.700243,0.686076,0.745819
3,0.640941,0.631675,0.647570,0.654326
4,0.638054,0.628247,0.655941,0.646610
...,...,...,...,...
98,0.166078,0.153730,0.227547,0.224171
99,0.156419,0.143823,0.215822,0.208432
100,0.188339,0.162471,0.228799,0.209106
101,0.182803,0.155128,0.233580,0.213041


In [18]:
train_data.shape

(103, 4)

In [19]:
test_data.shape

(30, 4)

In [20]:
for i in range(1,11):
    model=VAR(train_data)
    results=model.fit(i)
    print('Order=',i)
    print('AIC=',results.aic)
    print('BIC=',results.bic)

Order= 1
AIC= -28.18034897338623
BIC= -27.665648421761862
Order= 2
AIC= -27.965668810055472
BIC= -27.033546645636807
Order= 3
AIC= -27.8401923841987
BIC= -26.485503887484892
Order= 4
AIC= -27.94921603932423
BIC= -26.16670947559542
Order= 5
AIC= -27.754043834219132
BIC= -25.538357423930073
Order= 6
AIC= -27.479761305599123
BIC= -24.82542009064718
Order= 7
AIC= -27.438604687776124
BIC= -24.34001728975249
Order= 8
AIC= -27.42053314637915
BIC= -23.871988412786816
Order= 9
AIC= -27.270566102601602
BIC= -23.266229636899894
Order= 10
AIC= -27.10922780323184
BIC= -22.643138374445453


In [21]:
x=model.select_order(maxlags=10)
order=x.selected_orders['aic']
order

1

In [22]:
result=model.fit(order)
result

In [23]:
lagged_values=train_data.values[-order:]
lagged_values

array([[0.15936387, 0.12377617, 0.20717122, 0.1860596 ]])

In [24]:
prediction=result.forecast(y=lagged_values,steps=30)
prediction

array([[0.14274383, 0.13701333, 0.17927191, 0.18631389],
       [0.14309878, 0.1317854 , 0.17467907, 0.1713002 ],
       [0.14241138, 0.13147554, 0.17417337, 0.17050429],
       [0.14201927, 0.13093013, 0.17366093, 0.16974155],
       [0.14160232, 0.13051802, 0.1732683 , 0.16933875],
       [0.14119975, 0.13010309, 0.17287848, 0.16893452],
       [0.14080234, 0.12969793, 0.17249756, 0.16854561],
       [0.14041121, 0.12929862, 0.1721223 , 0.16816221],
       [0.14002601, 0.1289055 , 0.17175284, 0.16778489],
       [0.13964668, 0.12851835, 0.17138899, 0.16741331],
       [0.13927313, 0.1281371 , 0.17103069, 0.16704739],
       [0.13890527, 0.12776166, 0.17067785, 0.16668705],
       [0.13854302, 0.12739194, 0.17033039, 0.1663322 ],
       [0.13818628, 0.12702785, 0.16998822, 0.16598275],
       [0.13783498, 0.12666931, 0.16965126, 0.16563863],
       [0.13748904, 0.12631623, 0.16931944, 0.16529975],
       [0.13714836, 0.12596853, 0.16899267, 0.16496603],
       [0.13681287, 0.12562613,

In [25]:
prediction.shape

(30, 4)

In [26]:
pred_table=pd.DataFrame(prediction,columns=list)
pred_table

,Close,High,Low,Open
0,0.142744,0.137013,0.179272,0.186314
1,0.143099,0.131785,0.174679,0.171300
2,0.142411,0.131476,0.174173,0.170504
3,0.142019,0.130930,0.173661,0.169742
4,0.141602,0.130518,0.173268,0.169339
5,0.141200,0.130103,0.172878,0.168935
6,0.140802,0.129698,0.172498,0.168546
7,0.140411,0.129299,0.172122,0.168162
8,0.140026,0.128905,0.171753,0.167785
9,0.139647,0.128518,0.171389,0.167413


In [27]:
rmse=round(mean_squared_error(test_data,prediction,squared=False))
mape=mean_absolute_percentage_error(test_data,prediction)

In [28]:
performance['Model'].append(list)
performance['RMSE'].append(rmse)
performance['MaPe'].append(mape)
performance['Lag'].append(order)
performance['Test'].append(test_obs)

In [29]:
performance

{'Model': [['Close', 'High', 'Low', 'Open']],
 'RMSE': [0],
 'MaPe': [0.5008672121508082],
 'Lag': [1],
 'Test': [30]}

In [30]:
perf_table=pd.DataFrame(performance)
perf_table

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Low, Open]",0,0.500867,1,30


In [31]:
combinations=[['Close','High'],['Close','High','Low'],['Close','High','Low','Open']]
for columns in combinations:
    print('\nFor the combination of ',columns)
    #Manually fits the VAR model using training data
    for i in range(1,11):
        model=VAR(train_data[columns])
        results=model.fit(i)
        print('Order=',i)
        print('AIC=',results.aic)
        print('BIC=',results.bic)
    #Automatically selecting the lag order based on AIC
    x=model.select_order(maxlags=10)
    order=x.selected_orders['aic']
    result=model.fit(order)
    #Forecasting the future values equivalent to test set
    lagged_values=train_data[columns].values[-order:]
    prediction=result.forecast(y=lagged_values,steps=30)
    pred_table=pd.DataFrame(prediction,columns=columns)
    print('\nThe corresponding predicted values: ')
    print(pred_table)
    #Calculating the test metrics
    rmse=round(mean_squared_error(test_data[columns],prediction,squared=False))
    mape=mean_absolute_percentage_error(test_data[columns],prediction)
    #Appending the corresponding values to the performance table
    performance['Model'].append(columns)
    performance['RMSE'].append(rmse)
    performance['MaPe'].append(mape)
    performance['Lag'].append(order)
    performance['Test'].append(test_obs)
    perform_table=pd.DataFrame(performance)
print('\nThe performance table:')
print(perform_table)


For the combination of  ['Close', 'High']
Order= 1
AIC= -12.89067246429035
BIC= -12.736262298803041
Order= 2
AIC= -12.875123740854947
BIC= -12.616200917405319
Order= 3
AIC= -12.886447337074928
BIC= -12.521723511036594
Order= 4
AIC= -12.86079395596079
BIC= -12.388953983209047
Order= 5
AIC= -12.803563525756905
BIC= -12.22326470401453
Order= 6
AIC= -12.733607272046475
BIC= -12.04347855615897
Order= 7
AIC= -12.712054621477414
BIC= -11.910695811643714
Order= 8
AIC= -12.656473500944971
BIC= -11.742454402898462
Order= 9
AIC= -12.58958576820542
BIC= -11.561445324309036
Order= 10
AIC= -12.570378993546743
BIC= -11.426624383735595

The corresponding predicted values: 
       Close      High
0   0.147890  0.141673
1   0.149113  0.137433
2   0.147995  0.137310
3   0.147325  0.136437
4   0.146584  0.135716
5   0.145868  0.134980
6   0.145160  0.134259
7   0.144462  0.133547
8   0.143775  0.132845
9   0.143097  0.132154
10  0.142429  0.131473
11  0.141771  0.130801
12  0.141122  0.130139
13  0.14048